<img width="250" src="logo.png">

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import sys
from config import *

In [4]:
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials
from time import sleep

In [7]:
#Initialize SpotiPy with user credentias
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= client_id,
                                                           client_secret= client_secret))

In [8]:
final = pd.read_csv('final.csv')

In [9]:
final.head()

,song,artist,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,...,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature,hot,KMeans_13,KMeans_6
0,Flowers,Miley Cyrus,0.707,0.681,0.0,-4.325,1.0,0.0668,0.0632,0.000005,...,117.999,audio_features,spotify:track:0yLdNVWF3Srea0uzk55zFn,https://api.spotify.com/v1/tracks/0yLdNVWF3Sre...,https://api.spotify.com/v1/audio-analysis/0yLd...,200455.0,4.0,Yes,11,3
1,Kill Bill,SZA,0.644,0.728,8.0,-5.750,1.0,0.0351,0.0543,0.169000,...,88.993,audio_features,spotify:track:3OHfY25tqY28d16oZczHc8,https://api.spotify.com/v1/tracks/3OHfY25tqY28...,https://api.spotify.com/v1/audio-analysis/3OHf...,153947.0,4.0,Yes,0,3
2,Last Night,Morgan Wallen,0.517,0.675,6.0,-5.382,1.0,0.0357,0.4590,0.000000,...,203.853,audio_features,spotify:track:59uQI0PADDKeE6UZDTJEe8,https://api.spotify.com/v1/tracks/59uQI0PADDKe...,https://api.spotify.com/v1/audio-analysis/59uQ...,163855.0,4.0,Yes,4,2
3,Unholy,Sam Smith & Kim Petras,0.719,0.454,2.0,-7.384,1.0,0.0707,0.0138,0.000005,...,131.007,audio_features,spotify:track:3bXRd0yVb5sw2WVblM5URR,https://api.spotify.com/v1/tracks/3bXRd0yVb5sw...,https://api.spotify.com/v1/audio-analysis/3bXR...,156943.0,4.0,Yes,11,2
4,Creepin',"Metro Boomin, The Weeknd & 21 Savage",0.715,0.620,1.0,-6.005,0.0,0.0484,0.4170,0.000000,...,97.950,audio_features,spotify:track:2dHHgzDwk4BJdRwy9uXhTO,https://api.spotify.com/v1/tracks/2dHHgzDwk4BJ...,https://api.spotify.com/v1/audio-analysis/2dHH...,221520.0,4.0,Yes,11,3


In [29]:
def get_user_song():
    artist = input("Enter artist: ")
    title  = input("Enter title: ")
    return artist,title

In [35]:
def get_song_id(artist, title):
    string = "track:" + title + " artist:" + artist
    try:
        results = sp.search(string, limit=1)
        return results['tracks']['items'][0]['id']
    except:
        return None

In [68]:
def recommender():

    # 0) Load dataset
    songs = pd.read_csv("../data/clean/final.csv")
    
    # 1) Load scaler (stdscaler)
    with open("../scalers/scaler.pickle", "rb") as file:
        scaler = pickle.load(file)
    
    # 2) Load model (kmeans)
    with open("../models/kmeans_6.pickle", "rb") as file:
        kmean6 = pickle.load(file)
    
    while True:
        # 3) User enters artist and title
        artist,title = get_user_song()
        
        # 4) Request song id from spotify
        song_id = get_song_id(artist, title)
        if not song_id:
            print("! Unable to get song id")
            continue
            
        # 5) Request audio features using song id
        audio_feat = sp.audio_features(song_id)[0]
        if not audio_feat:
            print("! Unable to get audio features")
            continue
            
        for key,value in audio_feat.items():
            audio_feat[key] = [value]
            
            
        
        # 6) scale audio features
        audio_feat_df = pd.DataFrame(audio_feat).drop(['type','uri','track_href',
                                                       'analysis_url','id',
                                                      'mode','duration_ms','time_signature'],axis=1)

        audio_feat    = scaler.transform(audio_feat_df)
        audio_feat_df = pd.DataFrame(audio_feat, columns=audio_feat_df.columns)
        
        # 7) predict cluster using scaled audio features
        cluster = kmean6.predict(audio_feat_df)[0]

        # 8.1) if user song is a hot song
        if not songs[(songs["artist"] == artist) & (songs["song"] == title) & (songs["hot"] == "Yes")].empty:
            recommended = songs[(songs["hot"] == "Yes") & (songs["KMeans_6"] == cluster)].sample()
        else:
            recommended = songs[(songs["hot"] == "No") & (songs["KMeans_6"] == cluster)].sample()
    
        artist = recommended["artist"].values[0]
        title  = recommended["song"].values[0]
        
        print("Recommendation: {} - {}".format(artist, title))
        
    
        conti = input("\nDo you want to continue? (y/n) ")
        if conti == "n":
            break
        
        conti = input("\nDo you want to try our new feature, we know what you wanna roll!! (y/n) ")
        if conti == 'y':
            mood = input("\nHow do you feel today? How is your mood and what you wanna hear?")
            choose = input("\nPlease choose one number :\n1 Stressed at work and want to relax?\
                         \n2 On a mood of a lazy Sunday?\
                         \n3 Want to start the day with full of energy that anybody can't stop you?\
                         \n4 Friday night? Do you wanna dance?\
                         \n5 How about to sing with family or friends today? Or instead you can just sing in the shower!!\
                         \n6 Do you like good insturmentals? Want to ear a guitar screaming or a awesome bass line, or instead feel a living drums?\
                         \n7 Do you wanna quit? Please type 7.")
                        
        elif conti == 'n':
            break

        if choose =='1':
            recommended1 = songs[(songs["KMeans_6"] == 0)].sample()
            artist1 = recommended1["artist"].values[0]
            title1  = recommended1["song"].values[0]
            print("\n\nRecommendation: {} - {}".format(artist1, title1))
            

        elif choose =='2':
            recommended2 = songs[(songs["KMeans_6"] == 1)].sample()
            artist2 = recommended2["artist"].values[0]
            title2  = recommended2["song"].values[0]
            print("\n\nRecommendation: {} - {}".format(artist2, title2))
            

        elif choose =='3':
            recommended3 = songs[(songs["KMeans_6"] == 2)].sample()
            artist3 = recommended3["artist"].values[0]
            title3  = recommended3["song"].values[0]
            print("\n\nRecommendation: {} - {}".format(artist3, title3))
            

        elif choose =='4':
            recommended4 = songs[(songs["KMeans_6"] == 3)].sample()
            artist4 = recommended4["artist"].values[0]
            title4  = recommended4["song"].values[0]
            print("\n\nRecommendation: {} - {}".format(artist4, title4))
            

        elif choose =='5':
            recommended5 = songs[(songs["KMeans_6"] == 4)].sample()
            artist5 = recommended5["artist"].values[0]
            title5  = recommended5["song"].values[0]
            print("\n\nRecommendation: {} - {}".format(artist5, title5))
          

        elif choose =='6':
            recommended6 = songs[(songs["KMeans_6"] == 5)].sample()
            artist6 = recommended6["artist"].values[0]
            title6  = recommended6["song"].values[0]
            print("\n\nRecommendation: {} - {}".format(artist6, title6))
            
                           
        conti = input("\nDo you want to continue? (y/n) ")
        if conti == "n":
            break
        

In [ ]:
recommender()

Enter artist: lukas
Enter title: lukas
Recommendation: Elvis Presley - Jailhouse Rock

Do you want to continue? (y/n) y

Do you want to try our new feature, we know what you wanna roll!! (y/n) y

How do you feel today? How is your mood and what you wanna hear?great

Please choose one number :
1 Stressed at work and want to relax?                         
2 On a mood of a lazy Sunday?                         
3 Want to start the day with full of energy that anybody can't stop you?                         
4 Friday night? Do you wanna dance?                         
5 How about to sing with family or friends today? Or instead you can just sing in the shower!!                         
6 Do you like good insturmentals? Want to ear a guitar screaming or a awesome bass line, or instead feel a living drums?                         
7 Do you wanna quit? Please type 7.1


Recommendation: The Beatles - A Day In The Life - Remastered 2009

Do you want to continue? (y/n) y
